<a href="https://colab.research.google.com/github/hopeyen/allocation_solvers/blob/main/allocation_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation 
!pip3 install gql
!pip3 install ortools
!pip install gurobipy

     |████████████████████████████████| 252 kB 11.4 MB/s 
     |████████████████████████████████| 179 kB 57.3 MB/s 
     |████████████████████████████████| 14.5 MB 89 kB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
     |████████████████████████████████| 131 kB 59.9 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.2.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.14.1 which is incompatible.


     |████████████████████████████████| 11.1 MB 5.8 MB/s 


In [ ]:
# Import
from pprint import pprint
from getpass import getpass
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from IPython.display import JSON, IFrame
import requests
import numpy as np
from scipy.optimize import minimize

import pandas as pd

import gurobipy as gp
from gurobipy import GRB


Subgraph Allocation Calculator 

Needed information: A list of subgraphs (for now use all), 

1. Query subgraph information (current signal and allocation)
2. Query indexer information (available allocation)
3. Set up the variables (would like to check the equations)
4. Use Optimizer to solve

In [ ]:
# Set up client
api_url = 'https://gateway.thegraph.com/network'

token = getpass()

client = Client(
    transport=RequestsHTTPTransport(api_url, use_json=True, headers=dict(Authorization=f"token {token}")),
    fetch_schema_from_transport=True,
)

··········


In [ ]:
def query(query_string):
  query_gql = gql(query_string)
  result = client.execute(query_gql)
  return result

In [ ]:
subgraphs_query = """query getSubgraphs {
        subgraphs(
          first: 1000
          skip: 0
          orderBy: displayName
          orderDirection: asc
        ) {
          id
          displayName
          owner {
            id
          }
          currentVersion {
            id
            createdAt
            subgraphDeployment {
              id
              ipfsHash
              signalledTokens
              stakedTokens
              indexingRewardAmount
              queryFeesAmount
              deniedAt
              versions(first: 1000) {
                id
                subgraph {
                  id
                  currentVersion {
                    id
                    subgraphDeployment {
                      id
                    }
                  }
                }
              }
            }
          }
        }
      }
"""
subgraphs_data = query(subgraphs_query)

In [ ]:
indexers_query = """query getIndexers {
          lemnis: indexer(id: "0xc60d0c8c74b5d3a33ed51c007ebae682490de261"){
            allocatedTokens
          }
          
          prime1: indexer(id: "0x6ac85b9d834b51b14a7b0ed849bb5199e04c05c5"){
            allocatedTokens
          }
          
          prime2: indexer(id: "0x6125ea331851367716bee301ecde7f38a7e429e7"){
            allocatedTokens
          }
        }

"""
indexers_data = query(indexers_query)

In [ ]:
wanted_subgraphs = ['Sushi - Mainnet Exchange',
  'Sushi - MasterChef',
  'Livepeer',
  'Audius Network Mainnet',
  'UMA Mainnet Voting',
  'RAI Mainnet',
  'mStable Protocol',
  'PoolTogether v3_1_0',
  'Curve',
  'DODOEX V2',
  'Enzyme Finance',
  'Omen',
  'Radicle',
  'Sushi - SushiBar',
  'Yearn Vaults V2 Subgraph',
  'eip1155'  ,
  'AirSwap',
  'C.R.E.A.M.',
  'Decentraland Marketplace',
  'Badger DAO',
  'juicebox',
  'PoolTogether Governance',
  'Ethereum User LP Transactions V1',
  'Pickle Finance']



wanted_ipfs = ['QmWkVS3Uzr2WsTwvxtte2dpHbSYJSQ1bTQMVciKXCWx7TM',
'Qmf1RePazWgnPpJRcejQZjdisifKcxH7RykFkpKoBKLc5M',
'QmbHg6vAJRD9ZWz5GTP9oMrfDyetnGTr5KWJBYAq59fm1W',
'QmUVskWrz1ZiQZ76AtyhcfFDEH1ELnRpoyEhVL8p6NFTbR',
'QmbL5761GrNnT8X3Emeny6NbXQxDwXxdpiER2boASKBhFW',
'QmNrS2U5DHqn5DJiKKD1ZS4BpHeAGDsKjftHMwX8LqSZqv',
'Qmcgtsin741cNTtgnkpoDcY92GDK1isRG5F39FNEmEok4n']

In [ ]:
# print((subgraphs_data['subgraphs']))

# filter through name
subgraphs_data['subgraphs'] = (list(filter(lambda subgraph: subgraph['displayName'] in wanted_subgraphs, subgraphs_data['subgraphs'])))

# # filter through ipfs
# subgraphs_data['subgraphs'] = (list(filter(lambda subgraph: subgraph['currentVersion']['subgraphDeployment']['ipfsHash'] in wanted_ipfs, subgraphs_data['subgraphs'])))

In [ ]:
# get indexer constraint
# A_lemnis = float(indexers_data['lemnis']['allocatedTokens'])/1e18
# A_prime1 = float(indexers_data['prime1']['allocatedTokens'])/1e18
# A_prime2 = float(indexers_data['prime2']['allocatedTokens'])/1e18

A_prime1 = 11738784.84
A_lemnis = 187682701.36
A_prime2 = 227384535

A_limit = A_prime2

In [ ]:
# data processing and filter - subgraphs

# flatten name
for subgraph in subgraphs_data['subgraphs']:
  subgraph['currentVersion']['subgraphDeployment']['displayName'] = subgraph['displayName']

# process subg deployment, ignore the ones without a lower bound of stake or signal
lowerbound = 10 * 1e18
subgs = [subgraph['currentVersion']['subgraphDeployment'] for subgraph in subgraphs_data['subgraphs'] 
         if (int(subgraph['currentVersion']['subgraphDeployment']['stakedTokens']) >= lowerbound) 
            & (int(subgraph['currentVersion']['subgraphDeployment']['signalledTokens']) >= lowerbound) ]

# format into a dictionary with token factor         
subgraphs = {subg['id'] : 
              {'id': subg['id'], 
               'ipfsHash': subg['ipfsHash'], 
               'displayName': subg['displayName'], 
               'signalledTokens': (float(subg['signalledTokens'])/1e18), 
               'stakedTokens': (float(subg['stakedTokens'])/1e18)} for subg in subgs}

# for easy access later, get the list of ids and ipfs hashes
subgraph_ids = list(set([id for id, subg in subgraphs.items()]))
subgraph_ipfs_hashes = list(set([subg['ipfsHash'] for id, subg in subgraphs.items()]))

# also for easy access later
current_allocations = {subg['ipfsHash']: (subg['stakedTokens']) for id, subg in subgraphs.items()}
current_signals = {subg['ipfsHash']: (subg['signalledTokens']) for id, subg in subgraphs.items()}

# number of subgraphs
n = len(subgraphs)

# Model 1: with only 1 free variable

In [ ]:
factory = gp.Model('Allocation calculator')

x_allocate = factory.addVars(subgraph_ipfs_hashes, lb=0, name="Subgraph allocation") # amount allocated

Restricted license - for non-production use only - expires 2022-01-13


In [ ]:
#Allocation Capacity
AllocationCap = factory.addConstr((gp.quicksum(x_allocate)
                    == A_limit),
                   name = "Capacity")

In [ ]:
n

24

In [ ]:
#0. Objective Function
# Taylor serie
# a*c ÷ (b+c) = a * c / b - a * c**2 / b**2 + a * c**3 / b**3 - a * c**4 / b**4 + a * c**5 / b**5   

                  #  + current_signals[i] * x_allocate[i]**3 / current_allocations[i]**3 
                  #  - current_signals[i] * x_allocate[i]**4 / current_allocations[i]**4 
                  #  + current_signals[i] * x_allocate[i]**5 / current_allocations[i]**5


obj = gp.quicksum(( current_signals[i] * x_allocate[i] / current_allocations[i] 
                   - current_signals[i] * x_allocate[i] * x_allocate[i] / (current_allocations[i]*current_allocations[i])  )
               for i in subgraph_ipfs_hashes)

factory.setObjective(obj, GRB.MAXIMIZE)

# factory.Params.NonConvex=2

In [ ]:
factory.optimize()

# Print output
print("Available allocation: " + str(A_limit))
for x in x_allocate.items():
  if (x[1].x != 0): print (str(x[0]) + ":" + str(x[1].x))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1 rows, 24 columns and 24 nonzeros
Model fingerprint: 0x917ef82d
Model has 24 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  QObjective range [8e-12, 2e-10]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+08, 2e+08]
Presolve time: 0.01s
Presolved: 1 rows, 24 columns, 24 nonzeros
Presolved model has 24 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.25595452e+06  8.33871565e+06  1.38e+04 0.00e+00  8.95e+05     0s
   1   8.63457976e+04  1.40364047e+06  6.53e+02 0.00e+00  9.20e+04  

# Two free variables


In [ ]:
factory2 = gp.Model('Allocation calculator 2')

x_allocate_2 = factory2.addVars(subgraph_ipfs_hashes, lb=0, name="Subgraph allocation") # amount allocated
y_allocate = factory2.addVars(subgraph_ipfs_hashes, lb=0, name="Subgraph allocation opposite") # opposite
# cur_allocations = factory2.addVars(subgraph_ids, current_allocations, lb=0, name="current allocations") # current allocations
# cur_signals = factory2.addVars(subgraph_ids, current_signals, lb=0, name="current signals") # current signals

#Allocation Capacity
AllocationCap = factory2.addConstr((gp.quicksum(x_allocate_2)
                    <= A_limit),
                   name = "Capacity")

#Extra free variable equates
FreeVarEquates = factory2.addConstrs((y_allocate[i] * (current_allocations[i] + x_allocate_2[i]) == x_allocate_2[i] for i in subgraph_ipfs_hashes),
                   name = "Free")

#0. Objective Function
# Taylor serie
# a*c ÷ (b+c) = a*c*z where z = 1/(b+c) such that constrain is 1 = z*(b+c)
obj = gp.quicksum(( current_signals[i] * y_allocate[i]  )
               for i in subgraph_ipfs_hashes)

factory2.setObjective(obj, GRB.MAXIMIZE)

factory2.Params.NonConvex=2

factory2.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1 rows, 48 columns and 24 nonzeros
Model fingerprint: 0x41dc95b2
Model has 24 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+08]
  Objective range  [1e+04, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.00s
Presolved: 97 rows, 49 columns, 144 nonzeros
Presolved model has 24 bilinear constraint(s)
Variable types: 49 continuous, 0 integer (0 binary)

Root relaxation: objective 2.754143e+05, 33 iterations, 0.00 seconds

    Nodes    |    Current Node    | 

In [ ]:
# Print output
print("Available allocation: " + str(A_limit))
print(factory2.ObjVal)
for x in x_allocate_2.items():
  if (x[1].x != 0): print (str(x[0]) + ":" + str(x[1].x))

Available allocation: 227384535
220053.555809331
QmNrS2U5DHqn5DJiKKD1ZS4BpHeAGDsKjftHMwX8LqSZqv:3748387.856003058
QmRLE9ueEaDvBD57qsgUBANmyXwd7f8cybj8oTcVWC4KGb:5226970.574689268
Qmaz1R8vcv9v3gUfksqiS9JUz7K9G8S5By3JYn8kTiiP5K:21728925.25311621
QmTKXLEdMD6Vq7Nwxo8XAfnHpG6H1TzL1AGwiqLpoae3Pb:13928254.969153287
QmXU1g4ju2zNBmDanMBHHuwJyDRG7pCjTK9DdAPvXqfQc4:14853616.600749068
Qmf1RePazWgnPpJRcejQZjdisifKcxH7RykFkpKoBKLc5M:19081280.578844067
QmRhh7rFt3qxfRMTZvHRNK6jCobX4Gx5TkzWXhZkuj57w8:12361071.473240934
QmZsXh48bDVNVvrkweZRoGJrXbvbXzTE5M8ztZYc46dPRx:2137237.4142475245
QmTBxvMF6YnbT1eYeRx9XQpH4WvxTV53vdptCCZFiZSprg:8669502.026610851
QmTYMx6Ls53MfqAPYbPwMyt45KKzayznc2kxeDiwmgvtF9:5462695.095977578
QmZdsSbRwVD7VVVm5WGxZZC6HYvbjnFb4hcwvQ4fTs5bxA:8523487.24175055
QmWkVS3Uzr2WsTwvxtte2dpHbSYJSQ1bTQMVciKXCWx7TM:26268643.63363329
QmbJfBH9wtW1N81d2HtYZZgyNi1S98EkQbWnXB3jqNP9Qw:1533829.4323327597
QmUVskWrz1ZiQZ76AtyhcfFDEH1ELnRpoyEhVL8p6NFTbR:16841968.729029816
QmcKFhdu1pM4ycFmqiYV6XJv3tC9UDmBRQy

# Scipy version

In [ ]:
subgraph_list = list(value for index, value in subgraphs.items())

def objective(x):
  total = 0
  for index in range(len(subgraph_list)): 
    total -= ( current_signals[subgraph_list[index]['ipfsHash']] / (current_allocations[subgraph_list[index]['ipfsHash']] + x[index]) ) * x[index]
  # print("total: " + str(total))
  return total

def total_allocation_constraint(x):
  return A_limit - np.sum(x)

# initial guesses : A_limit / float(n)
n = len(subgraphs)
x0 = np.full(n, 0)

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
bnds = [(0.0, A_limit) for _ in range(n)]
con1 = {'type': 'eq', 'fun': total_allocation_constraint}
cons = ([con1])
solution = minimize(objective,x0,method='SLSQP',\
                    bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final SSE Objective: ' + str(objective(x)))

# print solution
print('Solution')
for i in range(n):
  if x[i] != 0:
    print(str(subgraph_list[i]['ipfsHash']) + ":" + str(x[i]))

Initial Objective: 0.0
Final SSE Objective: -12985.841384756322
Solution
QmbL5761GrNnT8X3Emeny6NbXQxDwXxdpiER2boASKBhFW:1676969.2629268821
QmbHg6vAJRD9ZWz5GTP9oMrfDyetnGTr5KWJBYAq59fm1W:1676969.2627961251
QmNrS2U5DHqn5DJiKKD1ZS4BpHeAGDsKjftHMwX8LqSZqv:1676969.2631033892
Qmcgtsin741cNTtgnkpoDcY92GDK1isRG5F39FNEmEok4n:1676969.2628100417
QmUVskWrz1ZiQZ76AtyhcfFDEH1ELnRpoyEhVL8p6NFTbR:1676969.262790388
QmWkVS3Uzr2WsTwvxtte2dpHbSYJSQ1bTQMVciKXCWx7TM:1676969.2627906944
Qmf1RePazWgnPpJRcejQZjdisifKcxH7RykFkpKoBKLc5M:1676969.2627824782


Extra

In [ ]:
# print("available allocation: " + str(A_limit))
# for subgraph in subgraphs:
#   print(subgraphs[subgraph])

import csv
csv_columns = ['id', 'ipfsHash','displayName','signalledTokens', 'stakedTokens']

csv_file = "current_subgraphs_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in subgraphs:
          writer.writerow(subgraphs[data])
except IOError:
    print("I/O error")

In [ ]:
2470627+1132547+2077132+1522565+1708770+2548964+278188

11738793